In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
import os




In [2]:
load_dotenv(override=True)

#print(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

True

In [3]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation")
model = ChatHuggingFace(llm=llm)

In [4]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [5]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [6]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [7]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [8]:
intial_state = {'title': 'write a beautiful poetry on spiritualism'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'write a beautiful poetry on spiritualism', 'outline': "Title: A Celestial Symphony: A Spiritual Poem\n\nI. Introduction\n   A. Preamble\n      1. The enchanting allure of the spirit realm\n      2. The desire to delve into the mystic\n\nII. The Ethereal journey\n   A. Ascension\n      1. A departure from the mundane\n      2. Embracing the first steps of the ethereal journey\n      3. An exploration of the indescribable phenomena, sights and sounds\n   B. Discovery\n      1. The revelation of divine energy\n      2. The unfathomable beauty of the celestial bodies\n      3. The wisdom hidden in the stars\n\nIII. The Inner Pilgrimage\n   A. Enlightenment\n      1. The illumination of the inner self\n      2. The vice versa effect of the spiritual journey on the person\n   B. Self-Discovery\n      1. The unfolding of inner truths\n      2. The understanding of the purpose and place in life\n      3. The acceptance and love for oneself\n\nIV. The Realization\n   A. Illumination\

In [9]:
print(final_state['outline'])

Title: A Celestial Symphony: A Spiritual Poem

I. Introduction
   A. Preamble
      1. The enchanting allure of the spirit realm
      2. The desire to delve into the mystic

II. The Ethereal journey
   A. Ascension
      1. A departure from the mundane
      2. Embracing the first steps of the ethereal journey
      3. An exploration of the indescribable phenomena, sights and sounds
   B. Discovery
      1. The revelation of divine energy
      2. The unfathomable beauty of the celestial bodies
      3. The wisdom hidden in the stars

III. The Inner Pilgrimage
   A. Enlightenment
      1. The illumination of the inner self
      2. The vice versa effect of the spiritual journey on the person
   B. Self-Discovery
      1. The unfolding of inner truths
      2. The understanding of the purpose and place in life
      3. The acceptance and love for oneself

IV. The Realization
   A. Illumination
      1. A true understanding of the self
      2. The illumination of the broader cosmos
   

In [10]:
print(final_state['content'])

write a beautiful spiritual poem titled "A Celestial Symphony: A Spiritual Poem" based on the provided outline

I. Introduction
   A. Preamble
      1. The enchanting allure of the spirit realm
      2. The desire to delve into the mystic

Embrace me, oh my celestial muse,
In your illuminated ethereal hues.
The allure, oh, eternal and profound,
Of your sacred realm, doth astound.

II. The Ethereal journey
   A. Ascension
      1. A departure from the mundane
      2. Embracing the first steps of the ethereal journey
      3. An exploration of the indescribable phenomena, sights and sounds

Away from earthly shackles, I rise,
In the moments' quiet respite,
A step closer to celestial realms,
And a world of truth that unfurls.

III. The Inner Pilgrimage
   A. Enlightenment
      1. The illumination of the inner self
      2. The vice versa effect of the spiritual journey on the person

My spirit illuminated,
A silhouette, pure and altruistic,
Explored at the core, I sought,
The origin and